In [ ]:
import os.path
import math
import rmgpy
import rmgpy.constants as constants
from IPython.display import display
from rmgpy.data.rmg import RMGDatabase
from rmgpy.thermo.nasa import NASA, NASAPolynomial
from rmgpy.species import Species
from rmgpy.chemkin import readThermoEntry, writeThermoEntry
from rmgpy.thermo.thermoengine import processThermoData
from rmgpy.data.thermo import findCp0andCpInf

In [ ]:
databasePath = rmgpy.settings['database.directory']

database = RMGDatabase()
database.load(
    path = databasePath,
    thermoLibraries = [],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsFamilies = 'none'
    )

In [ ]:
data = [
    """C2HBr             T04/04C  2.H  1.BR 1.   0.G   200.000  6000.000 1000.        1
 6.55399311E+00 3.37962726E-03-1.18362410E-06 1.87797808E-10-1.11059116E-14    2
 3.17495713E+04-8.20269727E+00 1.10795098E+00 3.21065018E-02-6.02244383E-05    3
 5.45400888E-08-1.86034151E-11 3.26428366E+04 1.67414085E+01 3.39671249E+04    4
    """,
    """C2HCl             T05/08C  2.H  1.CL 1.   0.G   200.000  6000.000 1000.        1
 6.52865585E+00 3.32425623E-03-1.14637403E-06 1.79972218E-10-1.05639468E-14    2
 2.51378884E+04-9.16499932E+00 1.25077097E+00 3.10939695E-02-5.78728028E-05    3
 5.20651866E-08-1.76611780E-11 2.59985454E+04 1.50044210E+01 2.73367422E+04    4
    """,
    """CL                J 6/82CL  1    0    0    0G   200.000  6000.000 1000.        1
 2.94658358E+00-3.85985408E-04 1.36139388E-07-2.17032923E-11 1.28751025E-15    2
 1.36970327E+04 3.11330136E+00 2.26062480E+00 1.54154399E-03-6.80283622E-07    3
-1.59972975E-09 1.15416636E-12 1.38552986E+04 6.57020799E+00 1.45891941E+04    4
    """,
    """BR                J 6/82BR  1    0    0    0G   200.000  6000.000 1000.        1
 0.20866945E+01 0.71459733E-03-0.27080691E-06 0.41519029E-10-0.23016335E-14    2
 0.12857696E+05 0.90837335E+01 0.24820782E+01 0.18570465E-03-0.64313029E-06    3
 0.84642045E-09-0.30137068E-12 0.12709455E+05 0.68740409E+01 0.13453589E+05    4
    """,
    """I                 J 6/82I   1    0    0    0G   200.000  6000.000 1000.        1
 2.61667712E+00-2.66010320E-04 1.86060150E-07-3.81927472E-11 2.52036053E-15    2
 1.20582790E+04 6.87896653E+00 2.50041683E+00-4.48046831E-06 1.69962536E-08    3
-2.67708030E-11 1.48927452E-14 1.20947990E+04 7.49816581E+00 1.28402035E+04    4
    """
]

thermo = []
for entry in data:
    thermo.append(readThermoEntry(entry)[1])

In [ ]:
base = thermo[0].toThermoData()
print base
base = subtractThermoData(base, thermo[3].toThermoData())
base = addThermoData(base, thermo[4].toThermoData())
print base

In [ ]:
spc = Species().fromSMILES('C#CCl')
display(spc)
findCp0andCpInf(spc, base)
spc.thermo = processThermoData(spc, base)
print writeThermoEntry(spc)

In [ ]:
base2 = thermo[1].toThermoData()
print base2
base2 = subtractThermoData(base2, thermo[2].toThermoData())
base2 = addThermoData(base2, thermo[4].toThermoData())
print base2

In [ ]:
def addThermoData(thermoData1, thermoData2):
    """
    Add the thermodynamic data `thermoData2` to the data `thermoData1`,
    and return `thermoData1`.
    """
    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] += thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si += thermoData2.H298.value_si
    thermoData1.S298.value_si += thermoData2.S298.value_si

    return thermoData1

def subtractThermoData(thermoData1, thermoData2):
    """
    Subtract the thermodynamic data `thermoData2` from the data `thermoData1`,
    and return `thermoData1`.
    """
    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] -= thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si -= thermoData2.H298.value_si
    thermoData1.S298.value_si -= thermoData2.S298.value_si

    return thermoData1